In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List
%matplotlib inline

In [2]:
# step 1 - read in the files
dimensions_matrix = pd.read_excel('./test_data/Dimensions_Matrix.xlsx')
cutoff_matrix = pd.read_excel('./test_data/cutoff_data.xlsx')
weigths_matrix = pd.read_excel('./test_data/Weights.xlsx')
demographics_matrix = pd.read_excel('./test_data/Demographics.xlsx')

In [3]:
def get_deprivation_matrix(dimensions_matrix: pd.DataFrame, cutoff_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the deprivation matrix (g0) based on dimensions and cutoff values.

    Args:
        dimensions_matrix (pd.DataFrame): DataFrame containing dimensions data.
        cutoff_matrix (pd.DataFrame): DataFrame containing cutoff values.

    Returns:
        pd.DataFrame: Deprivation matrix (g0).
    """
    # Get column names for the dataframes
    dimensions_column_names: List[str] = dimensions_matrix.columns

    # Create a copy of the dimensions matrix as the deprivation matrix
    deprivation_matrix: pd.DataFrame = dimensions_matrix.copy()

    # Generate values for the deprivation matrix based on dimensions and cutoff values
    for index, col_name in enumerate(dimensions_column_names):
        cutoff_value = cutoff_matrix.at[index, 'Cutoff']
        deprivation_matrix[col_name] = deprivation_matrix[col_name].apply(lambda value: 1 if value == cutoff_value else 0)

    return deprivation_matrix

In [4]:
def calculate_weighted_deprivation_matrix(deprivation_matrix: pd.DataFrame, weights_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the weighted deprivation matrix based on the deprivation matrix and weights.

    Args:
        deprivation_matrix (pd.DataFrame): Deprivation matrix (g0).
        weights_matrix (pd.DataFrame): DataFrame containing weights.

    Returns:
        pd.DataFrame: Weighted deprivation matrix.
    """
    # Iterate through columns of the deprivation matrix
    for index, col_name in enumerate(deprivation_matrix.columns):
        # Get the weight corresponding to the column
        weight = weights_matrix.at[index, 'Weight']
        
        # Multiply the values in the column by the weight
        deprivation_matrix[col_name] = deprivation_matrix[col_name] * weight
    
    return deprivation_matrix

In [5]:

def calculate_deprevation_scores(weighted_deprivation_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate deprivation scores based on the weighted deprivation matrix.

    Args:
        weighted_deprivation_matrix (pd.DataFrame): DataFrame containing weighted deprivation values.

    Returns:
        pd.DataFrame: DataFrame containing deprivation scores.
    """
    # Determine the number of rows in the weighted deprivation matrix
    number_of_rows = len(weighted_deprivation_matrix)

    # Initialize a DataFrame to store the deprivation scores
    data = {'Scores': [0] * number_of_rows}
    deprevation_scores = pd.DataFrame(data)

    # Calculate the deprivation score for each row in the weighted deprivation matrix
    for index in range(number_of_rows):
        deprevation_score = weighted_deprivation_matrix.iloc[index].sum()
        
        # Update the 'Scores' column in the deprivation scores DataFrame
        deprevation_scores.at[index, 'Scores'] = deprevation_score

    return deprevation_scores

In [6]:

def get_censored_vector(deprevation_scores_vector: pd.DataFrame, cutoff_score: int) -> pd.DataFrame:
    """
    Generate a censored vector based on deprivation scores and a cutoff score.

    Args:
        deprevation_scores_vector (pd.DataFrame): DataFrame containing deprivation scores.
        cutoff_score (int): Cutoff score used to censor deprivation scores.

    Returns:
        pd.DataFrame: Censored vector of deprivation scores.
    """
    # Create a copy of the deprivation scores vector
    censored_vector = deprevation_scores_vector.copy()

    # Iterate through the rows of the vector and apply censoring
    for index in range(len(censored_vector)):
        deprivation_score = censored_vector.at[index, 'Scores']

        # Apply censoring: If the deprivation score is below the cutoff, set it to 0
        censored_vector.at[index, 'Scores'] = deprivation_score if deprivation_score >= cutoff_score else 0

    return censored_vector

In [7]:

def get_head_count_ratio(censored_vector: pd.DataFrame) -> float:
    """
    Calculate the headcount ratio (H) based on a censored vector.

    The headcount ratio (H) represents the percentage of the population with a positive score in the censored vector.

    Args:
        censored_vector (pd.DataFrame): DataFrame containing censored deprivation scores.

    Returns:
        float: The headcount ratio as a percentage.
    """
    # Calculate the count of individuals with a positive score
    positive_count = len(censored_vector.loc[censored_vector["Scores"] > 0])

    # Calculate the total count of individuals in the censored vector
    total_count = len(censored_vector)

    # Calculate the headcount ratio as a percentage
    headcount_ratio = 100 * (positive_count / total_count)

    return headcount_ratio

In [8]:
def get_average_deprivation_score(censored_vector: pd.DataFrame, number_of_dimensions: int) -> float:
    """
    Calculate the average deprivation score for the Alkire-Foster method.

    The average deprivation score is the weighted average of deprivation scores for individuals classified as poor.

    Args:
        censored_vector (pd.DataFrame): DataFrame containing censored deprivation scores.
        number_of_dimensions (int): Total number of dimensions or indicators.

    Returns:
        float: The average deprivation score as a percentage.
    """
    # Calculate the sum of deprivation scores for individuals classified as poor
    scores_sum = censored_vector['Scores'].sum()

    # Calculate the total count of poor individuals
    total_poor_people = len(censored_vector.loc[censored_vector["Scores"] > 0])

    # Calculate the average deprivation score
    average_deprivation_score = (scores_sum / number_of_dimensions) / total_poor_people

    # Convert the average deprivation score to a percentage
    average_deprivation_score_percentage = 100 * average_deprivation_score

    return average_deprivation_score_percentage.round(2)

In [9]:
def get_adjusted_head_count_ratio(average_deprivation_score: float, headcount_ratio: float) -> float:
    """
    Calculate the adjusted headcount ratio for the Alkire-Foster method.

    The adjusted headcount ratio represents the product of the headcount ratio and the average deprivation score.

    Args:
        average_deprivation_score (float): The average deprivation score as a percentage.
        headcount_ratio (float): The headcount ratio as a percentage.

    Returns:
        float: The adjusted headcount ratio as a percentage.
    """
    # Convert percentages to ratios (divide by 100)
    average_deprivation_score /= 100
    headcount_ratio /= 100

    # Calculate the adjusted headcount ratio
    adjusted_headcount_ratio = 100 * headcount_ratio * average_deprivation_score

    return adjusted_headcount_ratio.round(2)


In [10]:
def calculate_subgroup_data(demographics_matrix: pd.DataFrame, censored: pd.DataFrame) -> list:
    """
    Calculate subgroup data based on the demographics matrix and censored data.

    Args:
        demographics_matrix (pd.DataFrame): DataFrame containing demographic information.
        censored (pd.DataFrame): DataFrame containing censored data.

    Returns:
        list: List of subgroups along with their corresponding data.
    """
    sub_groups_data = []

    for col in demographics_matrix.columns:
        sub_group_name = col
        sub_group_data = {}
        total_censored = len(censored[censored['Scores'] > 0])

        for value in demographics_matrix[col].unique():
            subgroup_censored_count = len(demographics_matrix[(demographics_matrix[col] == value) & (censored['Scores'] > 0)])
            percentage = subgroup_censored_count / total_censored
            sub_group_data[value] = percentage

        sub_groups_data.append([sub_group_name, sub_group_data])

    return sub_groups_data

In [16]:
df = get_deprivation_matrix(dimensions_matrix, cutoff_matrix)
df = calculate_weighted_deprivation_matrix(df, weigths_matrix)
scores = calculate_deprevation_scores(df)
censored = get_censored_vector(scores,2)
print(f"{get_head_count_ratio(censored):.2F}")
print(get_average_deprivation_score(censored, 9))
print(get_adjusted_head_count_ratio(get_average_deprivation_score(censored, 9), get_head_count_ratio(censored)))
for row in calculate_subgroup_data(demographics_matrix, censored):
    print(row)


64.47
34.55
22.27
['Faculty', {'EBE': 0.18625954198473282, 'Sci': 0.13587786259541984, 'HS': 0.10534351145038168, 'Hu': 0.3251908396946565, 'CLM': 0.24732824427480915}]
['Gender', {'Male': 0.3099236641221374, 'Other': 0.007633587786259542, 'Female': 0.6824427480916031}]
['Race', {'Indian': 0.07633587786259542, 'Coloured': 0.03969465648854962, 'White': 0.10076335877862595, 'Black': 0.7801526717557252, 'Asian': 0.0030534351145038168}]


In [15]:
sub_groups_data = []
for col in demographics_matrix.columns:
    sub_group_name = col
    sub_group_data = dict()
    total_censored = len(censored["Scores"] > 0)
    sum = 0
    for value in demographics_matrix[col].unique():
        subgroup_censored_count = len(
            demographics_matrix[
                (demographics_matrix[col] == value) & (censored["Scores"] > 0)
            ]
        )
        sub_group_data[value] = subgroup_censored_count / total_censored

    sub_groups_data.append([sub_group_name, sub_group_data])

for row in sub_groups_data:
    print(row)

['Faculty', {'EBE': 0.12007874015748031, 'Sci': 0.08759842519685039, 'HS': 0.06791338582677166, 'Hu': 0.20964566929133857, 'CLM': 0.1594488188976378}]
['Gender', {'Male': 0.19980314960629922, 'Other': 0.004921259842519685, 'Female': 0.43996062992125984}]
['Race', {'Indian': 0.04921259842519685, 'Coloured': 0.025590551181102362, 'White': 0.06496062992125984, 'Black': 0.5029527559055118, 'Asian': 0.001968503937007874}]
